# Validation of custom find_nearest_index function

`numpy.searchsorted` does not have an option to use a custom key, so I guess I will have to do this myself.

In [13]:
import math
import numpy as np

#old function
def find_nearest_index(array, value) -> int:
    '''
    Get the index of the nearest value in array to value

    :param array: list of numeric values
    :type iterable:
    :param value: value to search for
    :type numeric:
    :return: index of nearest value to value
    '''
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or
                    math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return idx-1
    else:
        return idx

#helper for new function
def _numericalCompare(lhs, rhs,
                      lkey = lambda x: x,
                      rkey = lambda x: x) -> int:
    if lkey(lhs) < rkey(rhs):
        return -1
    elif lkey(lhs) > rkey(rhs):
        return 1
    else:
        return 0

#proposed new function
def find_nearest_index_2(array, value,
                         arrKey = lambda x: x,
                         valueKey = lambda x: x,
                         comp = _numericalCompare) -> int:
    if comp(value, array[0], valueKey, arrKey) == -1:
        return 0
    if comp(value, array[-1], valueKey, arrKey) == 1:
        return len(array) - 1

    lo : int = 0
    hi : int = len(array) - 1

    while lo <= hi:
        mid: int = int((hi + lo) / 2)
        comp_temp: int = comp(value, array[mid], valueKey, arrKey)

        if comp_temp == -1:
            hi = mid - 1
        elif comp_temp == 1:
            lo = mid + 1
        elif comp_temp == 0:
            return mid
        else:
            raise RuntimeError('Invalid compare value!')

    return lo if (arrKey(array[lo]) - valueKey(value)) < (valueKey(value) - arrKey(array[hi])) else hi

In [38]:
#first create a bunch of arrays and values to use as test cases
import random

testLists = list()
testValue = list()
for i in range(100):
    testLists.append(sorted(random.sample(range(10,100), 10)))
    testValue.append(random.randint(0,150))

def runTests(values, lists, f):
    ret = list()
    for i, n in enumerate(values):
        #for lst in testLists:
        ret.append(f(lists[i], n))
    return ret

std = runTests(testValue, testLists, find_nearest_index)
test = runTests(testValue, testLists, find_nearest_index_2)

In [40]:
assert(len(std) == len(test))

results = [std[i] == test[i] for i in range(len(test))]
values = [(std[i], test[i]) for i in range(len(test))]

for i in range(len(test)):
    if not results[i]:
        print('[{}] {}, {} -> {}'.format(i, testValue[i], testLists[i], values[i]))


[84] 83, [18, 20, 21, 28, 42, 45, 50, 68, 69, 97] -> (9, 8)
[95] 82, [29, 49, 60, 63, 74, 77, 80, 84, 88, 94] -> (7, 6)


In [50]:
[x - 83 for x in testLists[84]]
[x - 82 for x in testLists[95]]
testLists[95][7]

84